In [1]:
#Generate BiPartites and save as Objects. 
import itertools as it
import sys
import json
import re
import collections
import os
path='../../data'


filenames = os.listdir(path)

count=0

AllDataPidTitleBipartite={}
AllDataPidTrackListBipartite={}
AllDataAlbumTrackSetBipartite={}
AllDataArtistTrackSetBipartite={}
AllDataTrackArtistBipartite={}
AllDataTrackAlbumBipartite={}
AllDataTrackNameBipartite={}
AllDataAlbumNameBipartite={}
AllDataAritstNameBipartite={}
AllDataPidDescriptionBipartite={}

for filename in sorted(filenames):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        fullpath = os.sep.join((path, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        for playlist in mpd_slice['playlists']:
            playlistId=str(playlist['pid'])
            playlistTracks=[]
            playlistTitle=playlist['name']
            for track in playlist['tracks']:
                trackId=track['track_uri']
                trackName=track['track_name']
                trackArtistId=track['artist_uri']
                trackArtistName=track['artist_name']
                trackAlbumId=track['album_uri']
                trackAlbumName=track['album_name']
                playlistTracks.append(trackId)
                AllDataAlbumTrackSetBipartite.setdefault(trackAlbumId,[]).append(trackId)
                AllDataArtistTrackSetBipartite.setdefault(trackArtistId,[]).append(trackId)
                AllDataTrackArtistBipartite[trackId]=trackArtistId
                AllDataTrackAlbumBipartite[trackId]=trackAlbumId
                AllDataTrackNameBipartite[trackId]=trackName
                AllDataAlbumNameBipartite[trackAlbumId]=trackAlbumName
                AllDataAritstNameBipartite[trackArtistId]=trackArtistName
            AllDataPidTitleBipartite[playlistId]=playlistTitle
            AllDataPidTrackListBipartite[playlistId]=playlistTracks
            if 'description' in playlist:
                AllDataPidDescriptionBipartite[playlistId]=playlist['description']
            count=count+1
            if count % 10000 ==0:
                print 'processed' + str(count)

processed10000
processed20000
processed30000
processed40000
processed50000
processed60000
processed70000
processed80000
processed90000
processed100000
processed110000
processed120000
processed130000
processed140000
processed150000
processed160000
processed170000
processed180000
processed190000
processed200000
processed210000
processed220000
processed230000
processed240000
processed250000
processed260000
processed270000
processed280000
processed290000
processed300000
processed310000
processed320000
processed330000
processed340000
processed350000
processed360000
processed370000
processed380000
processed390000
processed400000
processed410000
processed420000
processed430000
processed440000
processed450000
processed460000
processed470000
processed480000
processed490000
processed500000
processed510000
processed520000
processed530000
processed540000
processed550000
processed560000
processed570000
processed580000
processed590000
processed600000
processed610000
processed620000
processed630000
p

In [33]:
'''
>>AllDataPidTitleBipartite={}
>>AllDataPidTrackListBipartite={}
>>AllDataAlbumTrackSetBipartite={}
AllDataArtistTrackSetBipartite={}
AllDataTrackArtistBipartite={}
AllDataTrackAlbumBipartite={}
AllDataTrackNameBipartite={}
AllDataAlbumNameBipartite={}
AllDataAritstNameBipartite={}
AllDataPidDescriptionBipartite={}
'''
import pickle
fileObj = open('./BiPartites/AllDataArtistTrackSetBipartite.pkl', 'w')
pickle.dump(AllDataArtistTrackSetBipartite, fileObj)
fileObj.close()
import pickle
fileObj = open('./BiPartites/AllDataTrackArtistBipartite.pkl', 'w')
pickle.dump(AllDataTrackArtistBipartite, fileObj)
fileObj.close()
import pickle
fileObj = open('./BiPartites/AllDataTrackAlbumBipartite.pkl', 'w')
pickle.dump(AllDataTrackAlbumBipartite, fileObj)
fileObj.close()
import pickle
fileObj = open('./BiPartites/AllDataTrackNameBipartite.pkl', 'w')
pickle.dump(AllDataTrackNameBipartite, fileObj)
fileObj.close()
import pickle
fileObj = open('./BiPartites/AllDataAlbumNameBipartite.pkl', 'w')
pickle.dump(AllDataAlbumNameBipartite, fileObj)
fileObj.close()
import pickle
fileObj = open('./BiPartites/AllDataAritstNameBipartite.pkl', 'w')
pickle.dump(AllDataAritstNameBipartite, fileObj)
fileObj.close()
import pickle
fileObj = open('./BiPartites/AllDataPidDescriptionBipartite.pkl', 'w')
pickle.dump(AllDataPidDescriptionBipartite, fileObj)
fileObj.close()

In [2]:
len(AllDataPidTitleBipartite), len(AllDataPidTrackListBipartite), len(AllDataAlbumTrackSetBipartite), len(AllDataArtistTrackSetBipartite), len(AllDataTrackArtistBipartite), len(AllDataTrackAlbumBipartite), len(AllDataTrackNameBipartite), len(AllDataAlbumNameBipartite), len(AllDataAritstNameBipartite)

(1000000, 1000000, 734684, 295860, 2262292, 2262292, 2262292, 734684, 295860)

# Build Indri Ready Indexes

# Load Background Pids

In [3]:
#BuildIndriIndexs
#Build750BackgroundIndexes/
import pickle
#backgroundPids=pickle.load(open('SplitsInformation/backgroundDataPids.pkl','rb'))
backgroundPids=AllDataPidTitleBipartite.keys()

In [4]:
len(backgroundPids)

1000000

In [5]:
import codecs

def entry(docId, docContent):
    entry='<DOC>\n<DOCNO>'+docId+'</DOCNO>\n<TEXT>'+docContent+'\n</TEXT>\n</DOC>'
    return entry

def writeIndexToFile(indexDocsList,writePath):
    count=0
    errors=[]
    with codecs.open(writePath,"w", encoding='utf-8') as f:
        for line in indexDocsList:
            try:
                f.write(line +"\n")
                count=count+1
            except:
                errors.append(line)
    print count
    return errors

# Build Index Ready Doc - Pid as Document and Tracks of Playlist as Terms

In [6]:
#buildIndexPidAsDocTracksAsTerms

backgroundpidDocs=[]
for pid in backgroundPids:
    trackList=AllDataPidTrackListBipartite[pid]
    backgroundpidDocs.append(entry(str(pid), ' '.join([item.replace('spotify:track:','') for item in trackList])))

In [7]:
writeIndexToFile(backgroundpidDocs,'./All1MDataIndexes/All1MIndexPidAsDocTracksAsTerms.txt')

1000000


[]

In [8]:
len(backgroundpidDocs)

1000000

In [9]:

import itertools as it
import sys
import json
import re
import collections
import os
path='../../data'


filenames = os.listdir(path)

count=0

pid750=[]
AlbumTrackSetBipartite750={}
ArtistTrackSetBipartite750={}

for filename in sorted(filenames[:750]):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        fullpath = os.sep.join((path, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        for playlist in mpd_slice['playlists']:
            playlistId=str(playlist['pid'])
            pid750.append(playlistId)
            
            for track in playlist['tracks']:
                trackId=track['track_uri']
                trackName=track['track_name']
                trackArtistId=track['artist_uri']
                trackArtistName=track['artist_name']
                trackAlbumId=track['album_uri']
                trackAlbumName=track['album_name']
                
                AlbumTrackSetBipartite750.setdefault(trackAlbumId,[]).append(trackId)
                ArtistTrackSetBipartite750.setdefault(trackArtistId,[]).append(trackId)
            count=count+1
            if count % 10000 ==0:
                print 'processed' + str(count)

processed10000
processed20000
processed30000
processed40000
processed50000
processed60000
processed70000
processed80000
processed90000
processed100000
processed110000
processed120000
processed130000
processed140000
processed150000
processed160000
processed170000
processed180000
processed190000
processed200000
processed210000
processed220000
processed230000
processed240000
processed250000
processed260000
processed270000
processed280000
processed290000
processed300000
processed310000
processed320000
processed330000
processed340000
processed350000
processed360000
processed370000
processed380000
processed390000
processed400000
processed410000
processed420000
processed430000
processed440000
processed450000
processed460000
processed470000
processed480000
processed490000
processed500000
processed510000
processed520000
processed530000
processed540000
processed550000
processed560000
processed570000
processed580000
processed590000
processed600000
processed610000
processed620000
processed630000
p

In [13]:
len(pid750)

1000000

In [14]:
albumDocsNorm=[]
albumDocsNonNorm=[]
for albumid, tracks in AlbumTrackSetBipartite750.items():
    normTracks=list(set(tracks))
    albumDocsNonNorm.append(entry(albumid.strip(), ' '.join(list([item.replace('spotify:track:','') for item in tracks]))))
    albumDocsNorm.append(entry(albumid.strip(), ' '.join(list([item.replace('spotify:track:','') for item in normTracks]))))

In [15]:
writeIndexToFile(albumDocsNorm,'./All1MDataIndexes/1MIndexAlbumAsDocNormTracksSetAsTerms.txt')
writeIndexToFile(albumDocsNonNorm,'./All1MDataIndexes/1MIndexAlbumAsDocNonNormTracksListAsTerms.txt')

734684
734684


[]

In [16]:
artistDocsNorm=[]
artistDocsNonNorm=[]
for albumid, tracks in ArtistTrackSetBipartite750.items():
    normTracks=list(set(tracks))
    artistDocsNonNorm.append(entry(albumid.strip(), ' '.join(list([item.replace('spotify:track:','') for item in tracks]))))
    artistDocsNorm.append(entry(albumid.strip(), ' '.join(list([item.replace('spotify:track:','') for item in normTracks]))))

In [17]:
len(ArtistTrackSetBipartite750.items())

295860

In [18]:
writeIndexToFile(artistDocsNorm,'./All1MDataIndexes/1MIndexArtistsAsDocNormTracksSetAsTerms.txt')
writeIndexToFile(artistDocsNonNorm,'./All1MDataIndexes/1MIndexArtistsAsDocNonNormTracksListAsTerms.txt')

295860
295860


[]

# Normalize Text

In [28]:
import codecs

stop='../stopwords.txt'
emojiSynonyms='../emojiWords.txt'
stopList=[]
emojiMap={}
emojis=[]
f=codecs.open(stop,'r', encoding='utf-8')

for line in f.readlines():
    stopList.append(line.strip())

f=codecs.open(emojiSynonyms,'r', encoding='utf-8')
for line in f.readlines():
    emoji, meaning= line.strip().split()
    emojiMap[emoji.strip()]=' '.join(meaning.strip().split(','))
    emojis.append(emoji.strip())
        
def NormalizeDates(title):    
    fullDecades=['1930','1940','1950','1960','1970','1980','1990','2000','2010']
    truncDecades=['30','40','50','60','70','80','90','00','10']
    year=['2001','2002','2003','2004','2005','2006','2007','2008','2009','2011','2012','2013','2014','2015','2016','2017']
    truncYear=['02','03','04','05','06','07','08','09','11','12','13','14','15','16','17']
    seasons=['spring', 'summer','fall','winter','sommer', 'autumn', 'verano' ]
    months=['january','jan','february','feb','march','april','may','june','july','august','aug','spetember','sep','october','oct','novemeber','nov','december','dec']
    monthPairs=[('january','jan'),('february','feb'),('august','aug'),('september','sept'), ('september','sep'),('october','oct'),('november','nov'),('december','dec')]

    title=title.lower().strip()
    
    for decade in fullDecades:
        if decade in title:
            newTitle=''
            for word in title.split():
                if decade in word:
                    word=word.replace(decade+"'",decade)
                    word=word.replace(decade+"s",decade)
                    word=word.replace(decade,' '+decade+'s ')
                newTitle=newTitle+' '+word
            #print title+'------'+newTitle
            title=newTitle.strip()

    for truncDecade in truncDecades:
        if truncDecade in title and ('19'+truncDecade not in title) and ('20'+truncDecade not in title):
            newTitle=''
            for word in title.split():
                if truncDecade+"'" in word  or truncDecade+"s" in word or truncDecade+u"´s" in word or truncDecade+"'s" in word or truncDecade+" s " in word or truncDecade+"ies" in word:
                    word=word.replace(truncDecade+"'",truncDecade)
                    word=word.replace(truncDecade+"s",truncDecade)
                    word=word.replace(truncDecade+" s",truncDecade)
                    word=word.replace(truncDecade+u"´s",truncDecade)
                    word=word.replace(truncDecade+"ies",truncDecade)
                    if truncDecade in truncDecades[:7]:
                        century='19'
                    else:
                        century='20'
                    word=word.replace(truncDecade,' '+century+truncDecade+'s ')
                newTitle=newTitle+' '+word
            title=newTitle.strip()
    if title.strip() in ['70-80','70,80,90','90/00', '70 80 90', '90 00', '80 90', '80-90','90-00', '60-70', '70 80']:
        for truncDecade in truncDecades:
            if truncDecade in truncDecades[:7]:
                century='19'
            else:
                century='20'
            title=title.replace(truncDecade,' '+century+truncDecade+'s ' )
    title=title.strip()
    for yr in year:
        if yr in title:
            title=title.replace(yr,' '+yr+' ')
            
    for yr in truncYear:
        if yr in title and '20'+yr not in title:
            if "'"+yr in title:
                #print title
                title=title.replace("'"+yr,' 20'+yr+' ')
                #print title
            if yr+"'" in title:
                #print title
                title=title.replace(yr+"'",' 20'+yr+' ')
                #print title
            if '2k'+yr in title or '2K'+yr in title:
                #print title
                title=title.replace('2k'+yr,' 20'+yr+' ')
                #print title
            if '-'+yr in title :
                #print title
                title=title.replace('-'+yr,' 20'+yr+' ')
                #print title
        if yr in title and '20'+yr not in title:
            for season in seasons:
                if season in title:
                    #print title
                    title=title.replace(yr,' 20'+yr+' ')
                    #print title
            for month in months:
                if month in title and '20'+yr not in title:
                    #print title
                    title=title.replace(yr,' 20'+yr+' ')
                    #print '-'+title
    for month,shortMonth in monthPairs:
        newTitle=''
        if shortMonth in title and month not in title:
            #print title
            for word in title.split():
                if shortMonth== word:
                    newTitle=newTitle+' '+month
                else:
                    newTitle=newTitle+' '+word
            title=newTitle
            #print '-'+title
    title=title.strip()
    title=' '.join(title.split())
    return title

def handleEmojis(title):
    for emo in emojis:
        if emo in title:
            title=title.replace(emo,' ')
            title=title+' '+emojiMap[emo]+' '
            
    title=title.replace(u'\U0001f3fc','')
    title=title.replace(u'\U0001f3fd','')
    title=title.replace(u'\U0001f3fb','')
    title=title.replace(u'\U0001f3fe','')
    title=title.replace(u'\u200d','')
    title=title.replace(u'\ufe0f','')
    title=title.replace(u'oshrug','shrug')
    title=title.replace(u'\ufffd','')
    title=title.replace(u'\U0001f37b','')
    title=title.replace(u'\u200d','')
    title=title.replace(u'\u2640','')
    title=title.replace(u'\u2642','')
    title=title.replace(u'\U0001f3b6','')
    title=title.replace(u'\u2728','')
    title=title.replace(u'\U0001f449','')
    title=title.replace(u'\U0001f3ff','')
    title=title.replace(u'\U0001f38a','')
    title=title.replace(u'\U0001f445','')
    title=title.replace(u'\U0001f608','')
    title=title.replace(u'\U0001f381','')
    title=title.replace(u'\U0001f60f','')
    title=title.replace(u'\U0001f4a8','')
    title=title.replace(u'�','')
    title=title.replace('<3',' heart love ')
    title=title.replace(':)',' smile happy ')
    title=title.replace(';)',' smirk happy ')
    title=title.replace(':-)',' smile happy ')
    title=title.replace(': )',' smile happy ')
    title=title.replace(u'😋',' smile happy ')
    title=title.replace(u'\u263a\ufe0f',' smile happy ')
    title=title.replace('r&b',' randb ')
    title=title.replace('r & b',' randb ')
    title=title.replace('rnb',' randb ')
    title=title.replace(u'•',' ')
    title=title.replace(u'\u263a\ufe0f',' death poison ')
    title=title.replace(u'\u2615\ufe0f',' coffee tea morning ')
    title=title.replace(u'💩',' poop ')
    title=title.strip()
    title=' '.join(title.split())
    return title
# # ' & - $ . : ! / () * ,
import re
def normalize_nameTitle(name):
    name = name.lower()
    name = NormalizeDates(name)
    name = handleEmojis(name)
    name = name.replace('happysmile','happy smile')
    name = re.sub(r"[.,\/#\'?\&\-!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#\'?\&\-!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [29]:
normalize_nameTitle('normalize2016 me? :) <3 .....')

u'normalize 2016 me smile happy heart love'

In [32]:
import itertools as it
import sys
import json
import re
import collections
import os
path='../../data'


filenames = os.listdir(path)

count=0

pid750=[]

TrackIdTitle750={}
TitleTrackId750={}

TrackIdArtistName750={}
TrackIdAbumName750={}
TrackIdTrackName750={}

AlbumTrackSetBipartite750={}
ArtistTrackSetBipartite750={}

for filename in filenames:
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        fullpath = os.sep.join((path, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        for playlist in mpd_slice['playlists']:
            playlistId=str(playlist['pid'])
            pid750.append(playlistId)
            pname=playlist['name']
            normpName=normalize_nameTitle(pname).strip()
            if normpName=='':
                normpName='emptyTitle'
            for track in playlist['tracks']:
                trackId=track['track_uri']
                trackName=track['track_name']
                trackArtistId=track['artist_uri']
                trackArtistName=track['artist_name']
                trackAlbumId=track['album_uri']
                trackAlbumName=track['album_name']
                
                TrackIdTitle750.setdefault(trackId,[]).append(normpName)#--Done
                TitleTrackId750.setdefault(normpName,[]).append(trackId)#--Done
                
                TrackIdArtistName750[trackId]=trackArtistName#--meta2
                TrackIdAbumName750[trackId]=trackAlbumName#--meta2
                TrackIdTrackName750[trackId]=trackName#--meta2
                
                                
                AlbumTrackSetBipartite750.setdefault(trackAlbumId,[]).append(trackId)#done
                ArtistTrackSetBipartite750.setdefault(trackArtistId,[]).append(trackId)#done
                
            count=count+1
            if count % 10000 ==0:
                print 'processed' + str(count)

processed10000
processed20000
processed30000
processed40000
processed50000
processed60000
processed70000
processed80000
processed90000
processed100000
processed110000
processed120000
processed130000
processed140000
processed150000
processed160000
processed170000
processed180000
processed190000
processed200000
processed210000
processed220000
processed230000
processed240000
processed250000
processed260000
processed270000
processed280000
processed290000
processed300000
processed310000
processed320000
processed330000
processed340000
processed350000
processed360000
processed370000
processed380000
processed390000
processed400000
processed410000
processed420000
processed430000
processed440000
processed450000
processed460000
processed470000
processed480000
processed490000
processed500000
processed510000
processed520000
processed530000
processed540000
processed550000
processed560000
processed570000
processed580000
processed590000
processed600000
processed610000
processed620000
processed630000
p

In [33]:
len(TrackIdTitle750)

2262292

In [34]:
titleDocsNorm=[]
titletDocsNonNorm=[]
for title, tracks in TitleTrackId750.items():
    normTracks=list(set(tracks))
    titletDocsNonNorm.append(entry(title.strip(), ' '.join(list([item.replace('spotify:track:','') for item in tracks]))))
    titleDocsNorm.append(entry(title.strip(), ' '.join(list([item.replace('spotify:track:','') for item in normTracks]))))

In [35]:
err1=writeIndexToFile(titleDocsNorm,'./All1MDataIndexes/1MIndexTitlesAsDocNormTracksSetAsTerms.txt')
err2=writeIndexToFile(titletDocsNonNorm,'./All1MDataIndexes/1MIndexTitlesAsDocNonNormTracksListAsTerms.txt')

18303
18303


In [36]:
import nltk
def titlePlusBigrams(title):
    bigrm = list(nltk.bigrams(title.split()))
    bis=''
    for x1,x2 in bigrm:
        bis=bis+' '+x1+x2
    return title+' '+bis.strip()

In [37]:
len(titleDocsNorm)

18303

In [38]:
trackTitleDocs=[]
for trackId, titleList in  TrackIdTitle750.items():
    truncTrackId=trackId.replace('spotify:track:','')
    concatTitle=''
    for title in titleList:
        concatTitle=concatTitle+' '+titlePlusBigrams(title)
    trackTitleDocs.append(entry(truncTrackId.strip(), concatTitle))

    

In [39]:
err3=writeIndexToFile(trackTitleDocs,'./All1MDataIndexes/1MIndexTracksAsDocTitlesAsTerms.txt')

2262292


In [40]:
#TrackIdArtistName750[trackId]=trackArtistName#--meta2
#TrackIdAbumName750[trackId]=trackAlbumName#--meta2
#TrackIdTrackName750[trackId]=trackName#--meta2

meta2trackDocs=[]
for trackId, trackname in TrackIdTrackName750.items():
    truncTrackId=trackId.replace('spotify:track:','')
    normTrackName= normalize_nameTitle(trackname)
    normAlbumName= normalize_nameTitle(TrackIdAbumName750[trackId])
    normArtistName= normalize_nameTitle(TrackIdArtistName750[trackId])
    meta2trackDocs.append(entry(truncTrackId,normTrackName+' '+normAlbumName+' '+normArtistName))

In [41]:
err4=writeIndexToFile(meta2trackDocs,'./All1MDataIndexes/1MIndexTracksAsDocMeta2AsTerms.txt')

2262292


In [42]:
len(meta2trackDocs)

2262292

In [43]:
meta2trackDocs[:50]

[u'<DOC>\n<DOCNO>2d1caLRgavmHI07Hv6V3kM</DOCNO>\n<TEXT>el bazucaso en vivo desde hermosillo sonora 2007 20 corridos poderosos el tigrillo palma\n</TEXT>\n</DOC>',
 u'<DOC>\n<DOCNO>76ZRIwL6xXyPKtOIkUMAND</DOCNO>\n<TEXT>mine would be you modern day lullabies baby sleeptime\n</TEXT>\n</DOC>',
 u'<DOC>\n<DOCNO>0CbF79Ojec7oAzi0SJs7Cn</DOCNO>\n<TEXT>faded love countrypolitan piano the first four albums floyd cramer\n</TEXT>\n</DOC>',
 u'<DOC>\n<DOCNO>3VrBBMJMx9TZBUwoIf5MOO</DOCNO>\n<TEXT>high lifted everybody lift em a worshipper s perspective earnest pugh\n</TEXT>\n</DOC>',
 u'<DOC>\n<DOCNO>59LSmk4lneiptZrj8TZujq</DOCNO>\n<TEXT>nostalgia peste hierophant\n</TEXT>\n</DOC>',
 u'<DOC>\n<DOCNO>5h1ftnPNSUAFz0R8D8f2w4</DOCNO>\n<TEXT>dulce amor santa domingo vol 2 bachata band\n</TEXT>\n</DOC>',
 u'<DOC>\n<DOCNO>1y0dKXj8I27cYh1I15M5Bd</DOCNO>\n<TEXT>birdhouse booty feet booty feet\n</TEXT>\n</DOC>',
 u'<DOC>\n<DOCNO>5ckacg1TPKZKkUwfFmSote</DOCNO>\n<TEXT>about this thing about this thing young fran